In [ ]:
import os
from collections import defaultdict
from conllu import parse_incr

# Dictionnaire de mapping XPOS vers C8
xpos_to_c8 = {
    "WDT": "WPR",  # Relative pronoun, "that"
    "IN": "CST",   # Conjunction for nouns, "the fact that"
    "CJT": "CJT",  # Conjunction for verbs, "I think that"
    "DT": "DT",    # Singular determiner, "that"
    "RB": "RB"     # Adverb, "it's not that difficult"
}

def prepare_training_data_with_mapping(conllu_file, output_file, tag_mapping):
    """
    Prépare un corpus d'entraînement TreeTagger à partir d'un fichier CoNLL-U,
    en appliquant un mapping des tags XPOS vers un tagset C8 pour tous les mots.
    Compte également les occurrences de 'that' pour chaque catégorie.
    """
    training_data = []
    that_count = 0
    category_counts = defaultdict(int)
    
    with open(conllu_file, 'r', encoding='utf-8') as infile:
        for tokenlist in parse_incr(infile):  # Analyse incrémentale du fichier
            for token in tokenlist:
                word = token.get("form")
                lemma = token.get("lemma")
                xpos = token.get("xpos")
                
                # Appliquer le mapping des tags XPOS vers C8 si applicable
                c8_tag = tag_mapping.get(xpos, xpos)  # Utiliser le tag original si non mappé
                
                # Compter les occurrences de 'that' pour chaque catégorie
                if word.lower() == "that":
                    category_counts[c8_tag] += 1
                    that_count += 1
                
                # Ajouter la ligne formatée au corpus
                if word and lemma and xpos:
                    training_data.append(f"{word}\t{c8_tag}\t{lemma}")
            
            # Ajouter une ligne vide pour séparer les phrases
            training_data.append("")
    
    # Sauvegarder le corpus préparé
    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write("\n".join(training_data))
    
    print(f"Corpus d'entraînement sauvegardé dans : {output_file}")
    
    # Afficher le nombre total d'occurrences de 'that'
    print(f"Nombre total d'occurrences de 'that': {that_count}")
    
    # Afficher le nombre d'occurrences de 'that' pour chaque catégorie
    print("Nombre d'occurrences de 'that' pour chaque catégorie :")
    for category, count in category_counts.items():
        print(f"{category}: {count}")

# Fichiers d'entrée et de sortie
conllu_file = "training_data/en_partut-ud-train.conllu"  # Remplacez par le chemin de votre fichier d'entraînement
output_file = "corpus_mapped_that_c8_partut_xpos.txt"  # Chemin du fichier de sortie

# Préparer les données avec mapping global
prepare_training_data_with_mapping(conllu_file, output_file, xpos_to_c8)